## 赛题描述
本次大赛基于脱敏和采样后的数据信息，预测未来一段时间活跃的用户。

参考 

https://github.com/zhongqiangwu960812/AIGame/blob/master/GameOfKesci/%E5%BF%AB%E6%89%8B%E7%94%A8%E6%88%B7%E6%B4%BB%E8%B7%83%E5%BA%A6/%E5%BF%AB%E6%89%8B%E7%94%A8%E6%88%B7%E6%B4%BB%E8%B7%83%E9%A2%84%E6%B5%8B%E7%AC%94%E8%AE%B0%E7%89%88.ipynb


## 数据说明
大赛提供脱敏和采样后用户行为数据，日期信息进行统一编号，第一天编号为 01， 第二天为 02， 以此类推，所有文件中列使用 tab 分割。
一个二分类问题，即判断一个用户是否为活跃用户。

## 数据文件下载地址

链接：https://pan.baidu.com/s/1MhAU0zyMHA5-bUbPUjWRzQ 
提取码：dz9p

## 文件说明
###    app_launch_log.txt：用户登录数据集
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    day             String        日期                     01, 02 …30
    
###    user_activity_log.txt：用户行为数据集（对视频点赞，转发等）
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    day             String        日期                     01, 02 …30
    page            Int           行为发生的页面。         1
                                  每个数字分别对应
                                  “关注页”、“个人主页”
                                  “发现页”、“同城页”
                                  或“其他页”中的一个   
    video_id        Int           video id（脱敏后）       333
    author_id       Int           作者 id（脱敏后）        999
    action_type     Int           用户行为类型。           1
                                  每个数字分别对应
                                 “播放”、“关注”
                                 “点赞”、“转发”、
                                 “举报”和“减少此类作品”中的一个	

###    user_register_log.txt：用户注册数据集
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    register_day    String        日期                     01, 02 … 30
    register_type   Int           来源渠道（脱敏后）       0
    device type     Int           设备类型（脱敏后）       0
    
###    video_create_log.txt：视频创建数据集
    列名            类型           说明                    示例
    user_id         Int            用户唯一标识（脱敏后）  666
    day             String         拍摄日期                01, 02 … 30

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 数据集中使用制表符来进行分割
#注册数据
register = pd.read_csv('user_register_log.txt',sep='\t',names=['user_id','register_day','register_type','device_type'])
#登录数据
launch = pd.read_csv('app_launch_log.txt',sep='\t',names=['user_id','launch_day'])
#创建视频数据
create = pd.read_csv('video_create_log.txt',sep='\t',names=['user_id','create_day'])
#行为数据
activity = pd.read_csv('user_activity_log.txt',sep='\t',names=['user_id','act_day','page','video_id','author_id','act_type'])

In [3]:
print(register.shape)
print(launch.shape)
print(create.shape)
print(activity.shape)

(51709, 4)
(251943, 2)
(35151, 2)
(20607228, 6)


### 用户注册数据
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    register_day    String        日期                     01, 02 … 30
    register_type   Int           来源渠道（脱敏后）       0
    device type     Int           设备类型（脱敏后）       0

In [4]:
register.head()

,user_id,register_day,register_type,device_type
0,744025,1,1,283
1,1270299,1,1,259
2,571220,1,1,2
3,1308501,1,0,23
4,745554,1,2,0


### 用户登录数据

    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    day             String        日期                     01, 02 …30

In [5]:
#这个用户在哪些天登录了（每个用户可能不止一条）
launch.head()

,user_id,launch_day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


### 创建视频数据
    列名            类型           说明                    示例
    user_id         Int            用户唯一标识（脱敏后）  666
    day             String         拍摄日期                01, 02 … 30

In [6]:
#按照创建视频的时间记录，每个用户可能在同一天创建不止一个视频
create.head()

,user_id,create_day
0,720497,1
1,720497,1
2,720497,1
3,1075211,6
4,1075211,12


### 行为数据集    
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    day             String        日期                     01, 02 …30
    page            Int           行为发生的页面。         1
                                  每个数字分别对应
                                  “关注页”、“个人主页”
                                  “发现页”、“同城页”
                                  或“其他页”中的一个   
    video_id        Int           video id（脱敏后）       333
    author_id       Int           作者 id（脱敏后）        999
    action_type     Int           用户行为类型。           1
                                  每个数字分别对应
                                 “播放”、“关注”
                                 “点赞”、“转发”、
                                 “举报”和“减少此类作品”中的一个

In [7]:
activity.head()

,user_id,act_day,page,video_id,author_id,act_type
0,1062323,22,3,2877472,880271,0
1,639898,17,3,740662,210200,0
2,1260200,5,3,3332414,162866,0
3,817201,22,3,1129617,530246,0
4,817201,23,3,1129617,530246,0


    目标：预测第n+7天用户是否活跃
    模型：RNN
    Feature：  t1   ->  t2     ->   ...  ->   tn 天的数据
                |        |                     |   
    Label:     t(1+7)   t(2+7)       ...      t(n+7)天的活跃情况（是否登录）

In [8]:
#注册时间为1~31，用31-注册时间就可以得到该用户持续的时间
register['seq_length'] = 31 - register['register_day']
register.head()

,user_id,register_day,register_type,device_type,seq_length
0,744025,1,1,283,30
1,1270299,1,1,259,30
2,571220,1,1,2,30
3,1308501,1,0,23,30
4,745554,1,2,0,30


构建字典存储用户在持续时间内，不同日期的数据

In [9]:
user_queue = {i:[] for i in range(1,31)}
user_queue

{1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: []}

In [10]:
for index,row in register.iterrows():
    #row[-1]是seq_length,row[0]是user_id
    # 将user按照序列长度不同放到不同的数组中
    user_queue[row[-1]].append(row[0]) 

key表示用户持续的天数（序列长度），value表示用户id

In [11]:
user_queue[29]

[916655,
 719262,
 1026175,
 1140342,
 688100,
 1342459,
 926263,
 40710,
 246954,
 153579,
 200400,
 1195373,
 973391,
 65575,
 837576,
 298119,
 1358198,
 277129,
 566363,
 938471,
 854397,
 916651,
 563921,
 834452,
 291352,
 1178849,
 651257,
 445835,
 1144351,
 773513,
 485725,
 198615,
 194583,
 7327,
 372372,
 1105881,
 969026,
 667181,
 234084,
 649008,
 159024,
 1271674,
 415012,
 298599,
 811605,
 501579,
 860358,
 813291,
 296042,
 55692,
 256048,
 406412,
 1100342,
 392373,
 4615,
 824518,
 126911,
 1341370,
 736505,
 233989,
 145223,
 815305,
 371882,
 564434,
 1332694,
 182057,
 460251,
 299835,
 14401,
 1006192,
 582882,
 1360192,
 223179,
 777160,
 412321,
 457958,
 765739,
 90929,
 457810,
 896712,
 97843,
 51306,
 720211,
 133197,
 331962,
 28953,
 1326231,
 508771,
 20242,
 387201,
 48430,
 245662,
 1282923,
 458708,
 37980,
 1048096,
 567600,
 1103524,
 535350,
 70987,
 24609,
 150448,
 498695,
 1089620,
 929485,
 475036,
 660917,
 633625,
 1349886,
 368914,
 998039

In [12]:
class user_seq:
    
    def __init__(self,register_day,seq_length,n_features):
        self.register_day=register_day
        self.seq_length=seq_length
        # 构建全零的初始特征矩阵：持续天数*特征个数，后续新创建的特征来往里面填充
        # 特征个数固定为12
        # 持续天数，每个用户都不同，也就是针对不同的用户，根据他持续的天数，构建从 1x12 到 30 x 12的矩阵
        self.array=np.zeros([self.seq_length,n_features]) 
        self.array[0,0]=1
        self.page_rank=np.zeros([self.seq_length])
        self.pointer=1
        
    def put_feature(self,feature_number,string):
        # 每一天的记录用逗号','间隔，日期与状态之间用':'间隔
        for i in string.split(','):
            pos,value=i.split(':') #注册后第几天进行了登录，1为指示符
            self.array[int(pos)-self.register_day,feature_number]=1

    def put_PR(self,string):
        for i in string.split(','):
            pos,value=i.split(':')
            self.page_rank[int(pos)-self.register_day]=value

    def get_array(self):
        return self.array
    
    def get_label(self):
        #构造一个序列长度为seq_length，值为NaN的序列
        self.label=np.array([None]*self.seq_length)
        active=self.array[:,:10].sum(axis=1)  # 将用户的行为数据进行求和
        for i in range(self.seq_length-7): # 因为只有一个月30天的数据，用前7天预测第8天，所以最多只能预测30-7=23天的数据
            # 只要这个求和的数大于0，就说明这个用户在这一天是活跃的
            self.label[i]=1*(np.sum(active[i+1:i+8])>0)
        return self.label

In [13]:
# 每一个用户，每一天都统计12项特征
# 特征1：登录信息
n_features=12

# 针对注册数据register中的每一行构建字典
# 字典的key为row[0]: user_id
# 字典的value为一个类user_seq
data={row[0]:user_seq(register_day=row[1],seq_length=row[-1],n_features=n_features) for index,row in register.iterrows()}

In [14]:
data

{744025: <__main__.user_seq at 0x2bb1964d390>,
 1270299: <__main__.user_seq at 0x2bb1964d358>,
 571220: <__main__.user_seq at 0x2bb1964d3c8>,
 1308501: <__main__.user_seq at 0x2bb1964d400>,
 745554: <__main__.user_seq at 0x2bb1964d438>,
 1031012: <__main__.user_seq at 0x2bb1964d470>,
 913297: <__main__.user_seq at 0x2bb1964d4a8>,
 266500: <__main__.user_seq at 0x2bb1964d4e0>,
 475120: <__main__.user_seq at 0x2bb1964d518>,
 547944: <__main__.user_seq at 0x2bb1964d550>,
 916655: <__main__.user_seq at 0x2bb1964d588>,
 719262: <__main__.user_seq at 0x2bb1964d5c0>,
 1026175: <__main__.user_seq at 0x2bb1964d5f8>,
 1140342: <__main__.user_seq at 0x2bb1964d630>,
 688100: <__main__.user_seq at 0x2bb1964d668>,
 1342459: <__main__.user_seq at 0x2bb1964d6a0>,
 926263: <__main__.user_seq at 0x2bb1964d6d8>,
 40710: <__main__.user_seq at 0x2bb1964d710>,
 246954: <__main__.user_seq at 0x2bb1964d748>,
 153579: <__main__.user_seq at 0x2bb1964d780>,
 161418: <__main__.user_seq at 0x2bb1964d7b8>,
 649526:

### 处理登录信息

In [15]:
launch.head(5)

,user_id,launch_day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


In [16]:
launch.shape

(251943, 2)

In [17]:
# 里面没有重复的记录，意味着，每个用户，每一天登录只有一条记录
launch[launch.duplicated()]

,user_id,launch_day


In [18]:
launch['launch']=1 #每个用户每天登录次数
launch_table=launch.groupby(['user_id','launch_day'],as_index=False).agg({'launch':'sum'})
launch_table.head()

,user_id,launch_day,launch
0,16,13,1
1,16,14,1
2,16,15,1
3,16,18,1
4,16,19,1


In [19]:
def record_to_sequence(table): #得到用户特征序列表
    table.columns=['user_id','day','value']
    table.sort_values(by=['user_id','day'],inplace=True)
    table['string']=table.day.map(str)+':'+table.value.map(str)  # 构建新的字段'string', 表示为该日期登录，标识为1，以冒号':'分割
    # 将数据表按照user_id分组，并将各个string字段合并，以逗号','分割
    table=table.groupby(['user_id'],as_index=False).agg({'string':lambda x:','.join(x)})
    return table

In [20]:
launch_table=record_to_sequence(launch) #序列特征结果
launch_table.head()

,user_id,string
0,16,"13:1,14:1,15:1,18:1,19:1,20:1,21:1,22:1,23:1"
1,30,24:1
2,98,16:1
3,105,"12:1,14:1,15:1,16:1,17:1,18:1,19:1,20:1,21:1,2..."
4,176,"27:1,28:1,29:1,30:1"


例如ID=16的用户会在其特征表的指定位置的（13,14,15,18...行进行填充）

In [21]:
for index,row in launch_table.iterrows(): #根据登录信息对用户特征表进行填充
    # 在特征1位置上放入登录信息
    data[row[0]].put_feature(1,row[1]) #在指定特征位置上进行填充， row[0]: user_id， 

### 创作视频信息

In [22]:
create['create']=1
create.head()

,user_id,create_day,create
0,720497,1,1
1,720497,1,1
2,720497,1,1
3,1075211,6,1
4,1075211,12,1


In [23]:
create_table=create.groupby(['user_id','create_day'],as_index=False).agg({'create':'sum'})
create_table=record_to_sequence(create_table)
create_table.head()

,user_id,string
0,98,16:1
1,555,"22:1,24:1"
2,973,"13:1,14:3,20:1,25:1,27:4,28:5"
3,1014,"16:1,17:1,18:1,19:1,20:1,21:1"
4,1180,21:1


In [24]:
for index,row in create_table.iterrows():
    # 特征2：是否创作视频
    data[row[0]].put_feature(2,row[1])

#### 用户使用时行为特征，例如点赞，转发等


In [25]:
activity.head(5)

,user_id,act_day,page,video_id,author_id,act_type
0,1062323,22,3,2877472,880271,0
1,639898,17,3,740662,210200,0
2,1260200,5,3,3332414,162866,0
3,817201,22,3,1129617,530246,0
4,817201,23,3,1129617,530246,0


In [28]:
activity['act_type'].unique()

array([0, 2, 3, 1, 4, 5], dtype=int64)

In [29]:
#分别对不同行为进行统计，构建6种不同行为特征
for i in range(6):
    act=activity[activity.act_type==i].copy()
    act=act.groupby(['user_id','act_day'],as_index=False).agg({'video_id':'count'})
    act=record_to_sequence(act)
    for index,row in act.iterrows():
        #  将特征依次放到特征3-8中
        data[row[0]].put_feature(i+3,row[1])

#### 产生行为的界面信息

In [31]:
activity['page'].unique()

array([3, 1, 4, 0, 2], dtype=int64)

In [32]:
for i in range(1): #暂不作为特征，如果要做的话，只要改成range(5)就行
    act=activity[activity.page==i].copy()
    act=act.groupby(['user_id','act_day'],as_index=False).agg({'video_id':'count'})
    act=record_to_sequence(act)
    for index,row in act.iterrows():
        data[row[0]].put_feature(i+9,row[1]) 

#### 观看其他用户作品信息
user_id 不等于 author_id

In [34]:
activity.head(5)

,user_id,act_day,page,video_id,author_id,act_type
0,1062323,22,3,2877472,880271,0
1,639898,17,3,740662,210200,0
2,1260200,5,3,3332414,162866,0
3,817201,22,3,1129617,530246,0
4,817201,23,3,1129617,530246,0


In [35]:
watched=register.loc[:,['user_id']].copy()
watched.columns=['author_id']
# 取author_id != user_id 的交集
watched=pd.merge(watched,activity[activity.author_id!=activity.user_id],how='inner') #只得到交集，相当于看别人视频的
watched=watched.groupby(['author_id','act_day'],as_index=False).agg({'video_id':'count'})
watched=record_to_sequence(watched)
for index,row in watched.iterrows():
    # 特征10：观看其他人视频的信息
    data[row[0]].put_feature(10,row[1])

#### 观看自己的作品信息

In [36]:
watched=activity[activity.author_id==activity.user_id].copy()
watched=watched.groupby(['user_id','act_day'],as_index=False).agg({'video_id':'count'})
watched=record_to_sequence(watched)
for index,row in watched.iterrows():
    # 特征11：观看自己视频的信息
    data[row[0]].put_feature(11,row[1]) 

#### 制作数据标签
活跃用户定义为：在未来7天内使用过APP（在上述任一类型日志中出现过）


对用户从注册开始时进行统计，对于每1天的数据展开，如果其7天后仍有行为产生，则标签为1

In [39]:
label={user_id:user.get_label() for user_id,user in data.items()} 

In [40]:
label

{744025: array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, None, None, None, None, None, None, None], dtype=object),
 1270299: array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, None, None, None, None, None, None, None], dtype=object),
 571220: array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, None, None, None, None, None, None, None], dtype=object),
 1308501: array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, None, None, None, None, None, None, None], dtype=object),
 745554: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, None, None, None, None, None, None, None], dtype=object),
 1031012: array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, None, None, None, None, None, None, None], dtype=object),
 913297: array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1

#### 用户特征数据即为之前提取的data中的各项特征，转换成ndarray

In [41]:
data={user_id:user.get_array() for user_id,user in data.items()}

#### 将上面的方法合并到py文件中

In [44]:
register=pd.read_csv('user_register_log.txt',sep='\t',names=['user_id','register_day','register_type','device_type'])
launch=pd.read_csv('app_launch_log.txt',sep='\t',names=['user_id','launch_day'])
create=pd.read_csv('video_create_log.txt',sep='\t',names=['user_id','create_day'])
activity=pd.read_csv('user_activity_log.txt',sep='\t',names=['user_id','act_day','page','video_id','author_id','act_type'])


In [49]:
from deep_tools import DataGenerator

In [50]:
data_generator=DataGenerator(register,launch,create,activity)

#### 构建RNN

In [52]:
import tensorflow as tf

In [54]:
n_features = 12
n_hu = 8
with tf.variable_scope('train'):
    
    #变量与输入
    lr=tf.placeholder(tf.float32,[],name='learning_rate')

    W_out=tf.get_variable('W_out',[n_hu,1])
    b_out=tf.get_variable('b_out',[1])

    # x和y  x_shape(batch_size, seq_length, n_features)
    x=tf.placeholder(tf.float32,[None,None,n_features])
    y=tf.placeholder(tf.float32,[None,None])
    
    batch_size=tf.shape(x)[0]
    seq_length=tf.shape(x)[1]
    
    #RNN层
    cell=tf.nn.rnn_cell.GRUCell(n_hu)
    initial_state = cell.zero_state(batch_size, dtype=tf.float32)
    # 每一个batch长度相同，但是不同的batch，长度不一样，因此使用dynamic_rnn
    outputs, state = tf.nn.dynamic_rnn(cell, x,
                                       initial_state=initial_state)
     # 具体：https://blog.csdn.net/u010960155/article/details/81707498
        
    #输出层
    outputs=tf.reshape(outputs,[-1,n_hu])
    logits=tf.matmul(outputs,W_out)+b_out
    logits=tf.reshape(logits,tf.stack([batch_size,seq_length]))
   

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [55]:
#选择部分预测结果与标签当做训练损失计算
logits_local_train=logits[:,:-14]  # 这里-14或者是更小，因为本地训练，我们用前16天训练，16-23天测试
label_local_train=y[:,:-14]

In [56]:
# 设置损失函数

# 正则化项
regularizer = tf.contrib.layers.l2_regularizer(0.00001)
penalty = tf.contrib.layers.apply_regularization(regularizer, tf.trainable_variables())

obj_local = tf.losses.sigmoid_cross_entropy(label_local_train, logits_local_train) + penalty
optimizer = tf.train.AdamOptimizer(lr)
set_local = optimizer.minimize(obj_local)

# 选择部分预测结果与标签当做测试损失计算
logits_local_test = logits[:, -8]
label_local_test = y[:, -8]   #  这里也可以选择其他的天


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.


In [57]:
def train(n_obs=1000, step=1000, lr_feed=0.01):
    #date_seq = [31] + list(range(2, 16)) + [16] * 15
    variables = [set_local, obj_local, label_local_train, logits_local_train]
    
    for i in range(step):
        length, id_list, data_x, data_y = data_generator.next_batch(n_obs)
        _, los, lab, log = sess.run(variables, 
                                   feed_dict={x:data_x, y:data_y, lr:lr_feed})

In [58]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [59]:
train(n_obs=1000, step=2000, lr_feed=0.01)

In [60]:
def test():
    n_NA = 14     # 本地训练， 我们是1-16天训练，16-23天预测，所以这个地方最大是30-14
    # 优化目标和数据
    variables_1 = [obj_local, logits_local_train, label_local_train]
    variables_2 = [logits_local_test, label_local_test]
    
    obs_count, cum_loss, correct = 0, 0, 0
    user, prob, real = [], [], []
    
    # 训练损失
    for length, id_list, data_x, data_y in zip(*data_generator.get_set('train')):
        _obj, _logits_train, _label_train = sess.run(variables_1,
                                                    feed_dict={x:data_x, y:data_y, lr:0.001})
        obs_count += (length - n_NA) * len(id_list)
        cum_loss += _obj * (length - n_NA) * len(id_list)
        correct += np.sum((1 * (_logits_train>0) == _label_train))
    
    # 测试损失
    for length, id_list, data_x, data_y in zip(*data_generator.get_set('test')):
        _ = sess.run(variables_2, feed_dict={x:data_x, y:data_y, lr:0.001})
        _logits_test, _label_test = _
        real += list(_label_test)
        
        user += list(id_list)
        prob += list(1 / (1+np.exp(-_logits_test.reshape([-1]))))
    
    # 打印训练损失
    print('train_loss', cum_loss/obs_count)
    
    # 测试损失
    result = pd.DataFrame({'user_id':user, 'prob':prob, 'label':real})
    print('test_score:', f(result))
    
    return result

In [62]:
from deep_tools import f
test()

train_loss 0.46508258878152575
test_score: [0.804922850844967, 0.8045231693816226, 0.8040477893454132, 0.8037483266398929, 0.8033117389317491, 0.8034972205731556]


,user_id,prob,label
0,1274576,0.844183,1.0
1,109973,0.789269,1.0
2,134299,0.789269,0.0
3,1005835,0.576562,1.0
4,864582,0.792236,1.0
5,1293122,0.319002,1.0
6,896835,0.739193,1.0
7,630631,0.835431,1.0
8,1121791,0.739193,1.0
9,1038314,0.676065,1.0
